# Analyse de données sur une histoire du conflit politique

On va d'abord importer et nettoyer les données

Voici le code que j'ai fait pour transformer un csv de 700Mo en un fichier parquet plus léger qui ne contient que les colonnes nécessaires, pour le commit sur git

In [1]:
import polars as pl

# On prépare la requête sans charger les données
overrides = {
    "codecommune": pl.Categorical,
    "dep": pl.Categorical    
}

query = (
    pl.scan_csv("diplomescommunes.csv",
        schema_overrides=overrides, 
        separator=",")
    .select([
        "codecommune",
        "dep",
        "pbac2022",
        "psup2022"
        ])
)

df = query.collect()

print(df.head())
# Sauvegarde en Parquet (nécessite pyarrow, déjà installé avec polars souvent)
df.write_parquet("data_diplomes.parquet")

shape: (5, 4)
┌─────────────┬─────┬───────────┬───────────┐
│ codecommune ┆ dep ┆ pbac2022  ┆ psup2022  │
│ ---         ┆ --- ┆ ---       ┆ ---       │
│ cat         ┆ cat ┆ f64       ┆ f64       │
╞═════════════╪═════╪═══════════╪═══════════╡
│ 01001       ┆ 01  ┆ 0.4471404 ┆ 0.2322357 │
│ 01002       ┆ 01  ┆ 0.672414  ┆ 0.086207  │
│ 01003       ┆ 01  ┆ null      ┆ null      │
│ 01004       ┆ 01  ┆ 0.461044  ┆ 0.289937  │
│ 01005       ┆ 01  ┆ 0.337925  ┆ 0.168558  │
└─────────────┴─────┴───────────┴───────────┘


Ce code nous a servi à transformer des fichiers csv avec beaucoup de colonnes et de variables que nous ne souhaitons pas conserver dans notre analyse (données d'une année passée, etc...) en fichier parquet avec uniquement les variables qui nous intéressent

Nous allons maintenant faire un test avec quelques unes des données que nous avons collecté pour faire un tableau de données qui sera utile pour notre analyse

In [3]:
import polars as pl

# Liste des fichiers secondaires à ajouter
fichiers_a_joindre = [
    "data_revenu.parquet",
    "data_rsa.parquet",
    "data_diplomes.parquet"
]

# On part de la base
q_globale = pl.scan_parquet("data_vote2022.parquet")

# On boucle pour les ajouter un par un
for fichier in fichiers_a_joindre:
    q_autre = pl.scan_parquet(fichier)
    q_globale = q_globale.join(q_autre, on=["codecommune",'dep'], how="left")

df_complet = q_globale.collect()
print(df_complet.head())

shape: (5, 90)
┌─────┬─────────────┬─────────┬─────────┬───┬──────────┬────────────┬───────────┬───────────┐
│ dep ┆ codecommune ┆ voixAUG ┆ voixNUP ┆ … ┆ prsa2021 ┆ perrsa2021 ┆ pbac2022  ┆ psup2022  │
│ --- ┆ ---         ┆ ---     ┆ ---     ┆   ┆ ---      ┆ ---        ┆ ---       ┆ ---       │
│ cat ┆ cat         ┆ i64     ┆ i64     ┆   ┆ f64      ┆ f64        ┆ f64       ┆ f64       │
╞═════╪═════════════╪═════════╪═════════╪═══╪══════════╪════════════╪═══════════╪═══════════╡
│ 01  ┆ 01001       ┆ 4       ┆ 52      ┆ … ┆ 0.007653 ┆ 0.05651    ┆ 0.4471404 ┆ 0.2322357 │
│ 01  ┆ 01002       ┆ 4       ┆ 44      ┆ … ┆ 0.0      ┆ 0.030562   ┆ 0.672414  ┆ 0.086207  │
│ 01  ┆ 01004       ┆ 66      ┆ 1196    ┆ … ┆ 0.061711 ┆ 0.645217   ┆ 0.461044  ┆ 0.289937  │
│ 01  ┆ 01005       ┆ 5       ┆ 100     ┆ … ┆ 0.006663 ┆ 0.050262   ┆ 0.337925  ┆ 0.168558  │
│ 01  ┆ 01006       ┆ 3       ┆ 13      ┆ … ┆ 0.0      ┆ 0.038058   ┆ 1.0       ┆ 0.013158  │
└─────┴─────────────┴─────────┴─────────┴───┴